In [1]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
from copy import deepcopy


class BaseGraph():
    
    
    def __init__(self):
        
        self.iteration_counter = 0
        
        # Creating two subplots with two columns
        figure_obj = tools.make_subplots(rows=1, cols=2,
                                         subplot_titles=('Losses', 'Accuracy'))
        
        # Creating curves for val/train accuracy/loss curves
        training_loss_curve_trace = go.Scatter(
                                            x=[],
                                            y=[],
                                            name = 'Training loss')

        validation_loss_curve_trace = go.Scatter(
                                            x=[],
                                            y=[],
                                            name = 'Validation loss')

        training_accuracy_curve_trace = go.Scatter(
                                            x=[],
                                            y=[],
                                            name = 'Training accuracy')


        validation_accuracy_curve_trace = go.Scatter(
                                            x=[],
                                            y=[],
                                            name = 'Validation accuracy')

        # associating our scatter objects with each subplot
        figure_obj.append_trace(training_loss_curve_trace, 1, 1)
        figure_obj.append_trace(validation_loss_curve_trace, 1, 1)
        figure_obj.append_trace(training_accuracy_curve_trace, 1, 2)
        figure_obj.append_trace(validation_accuracy_curve_trace, 1, 2)

        #figure_obj['layout'].update(height=height,
        #                            width=width)
        
        # Plotly's Figure objects are not serializable,
        # therefore we convert them into ordered dict and serilize
        # later to store in the pickle format inside of our db.
        self.figure_obj = figure_obj.get_ordered()
        
        # Creating sym links for all curves,
        # so that we can easily update the object
        self.training_loss_history = self.figure_obj['data'][0]
        self.validation_loss_history = self.figure_obj['data'][1]
        self.training_accuracy_history = self.figure_obj['data'][2]
        self.validation_accuracy_history = self.figure_obj['data'][3]
    
    
    def add_next_iteration_results(self,
                                   training_loss,
                                   validation_loss,
                                   training_accuracy,
                                   validation_accuracy):
        
        
        self.training_loss_history['x'].append(self.iteration_counter)
        self.validation_loss_history['x'].append(self.iteration_counter)
        self.training_accuracy_history['x'].append(self.iteration_counter)
        self.validation_accuracy_history['x'].append(self.iteration_counter)
        self.iteration_counter += 1
        
        self.training_loss_history['y'].append(training_loss)
        self.validation_loss_history['y'].append(validation_loss)
        self.training_accuracy_history['y'].append(training_accuracy)
        self.validation_accuracy_history['y'].append(validation_accuracy)
        
        
        

In [1]:
import pebble
from time import sleep

process_pool = pebble.ProcessPool(max_tasks=1)

def dummy_func():
    
    #sleep(1000)
    return 100

In [2]:
future_obj = process_pool.schedule(dummy_func)

In [4]:
future_obj.exception()

In [15]:
from random import random

In [16]:
random()

0.8273446899669058

In [2]:
dummy = BaseGraph()

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [7]:
dummy.add_next_iteration_results(1, 1, 1, 1)
dummy.add_next_iteration_results(2, 2, 2, 2)
dummy.add_next_iteration_results(3, 3, 3, 3)

In [8]:
dummy.figure_obj

OrderedDict([('data',
              [OrderedDict([('x', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
                            ('y', [1, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]),
                            ('name', 'Training loss'),
                            ('type', 'scatter'),
                            ('xaxis', 'x1'),
                            ('yaxis', 'y1')]),
               OrderedDict([('x', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
                            ('y', [1, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]),
                            ('name', 'Validation loss'),
                            ('type', 'scatter'),
                            ('xaxis', 'x1'),
                            ('yaxis', 'y1')]),
               OrderedDict([('x', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
                            ('y', [1, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]),
                            ('name', 'Training accuracy'),
                            ('type', 'scatter'),
           

In [1]:
from wtforms import Form

from wtforms.ext.sqlalchemy.orm import model_form
from dash_deep.app import scripts_db_models
from dash_deep.widjets.widjets_factory import generate_widjet_from_form
from dash_deep.utils import generate_wtform_instances_and_input_form_widjets
from dash_deep.app import db
from dash_deep.models import EndovisBinary

from plotly.graph_objs import Figure

from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

from dash_deep.sql import create_dummy_endovis_records
from dash_deep.app import scripts_db_models

wtform_classes, scripts_input_form_widjets = generate_wtform_instances_and_input_form_widjets(scripts_db_models)

ImportError: cannot import name db

In [2]:
imagenet_form = wtform_classes[1]

In [3]:
imagenet_form.batch_size.data = 1000
imagenet_form.learning_rate.data = 0.0001

In [4]:
imagenet_form.data

{'batch_size': 1000, 'learning_rate': 0.0001}

In [5]:
sql_model = imagenet_form.sql_model_class()

imagenet_form.populate_obj(sql_model)

In [ ]:
db.engine.dispose

In [6]:
sql_model

<Imagenet classification experiment batch_size=1000, learning_rate=0.0001, graphs=None>

In [ ]:
from sqlalchemy.orm.attributes import flag_modified

class Experiment():
    
    def __init__(self, sql_model_instance):
        
        self.sql_model_instance = sql_model_instance
        
        # Since experiment will be most probably run in a separate,
        # process, we need to take appropriate actions to prevent different
        # processes from using the same connection:
        # http://docs.sqlalchemy.org/en/latest/core/pooling.html#using-connection-pools-with-multiprocessing
        db.engine.dispose()
        self.db = db
        
        # Making an initial commit to the database
        self.db.session.add(self.sql_model_instance)
        self.commit()
        
    def add_next_iteration_results(self, *args, **kwargs):
        
        # Passing all the parameters to the graph object
        # which upates the internal state of the graph
        self.sql_model_instance.graphs.add_next_iteration_results(*args, **kwargs)
        
        # Since we pickle this field -- we need to explicitly tell that
        # the field was updated, otherwise the changes won't be commited
        flag_modified(self.sql_model_instance, 'graphs')
        
        self.db.session.add(self.sql_model_instance)
        self.commit()
    
    def finish(self):
        
        # Compute the best results for each metric using graph object:
        # like highest accuracy + lowest loss -- and write them down
        # into database
        
        # Should we close the db session?
        
        self.db.session.add(self.sql_model_instance)
        self.commit()
        
        

In [12]:
def another(*args, **kwargs):
    
    print(args, kwargs)

def test(first, *args, **kwargs):
    
    print(first, args, kwargs)
    
    another(*args, **kwargs)

In [14]:
test(1, 100, name='danniil')

(1, (100,), {'name': 'danniil'})
((100,), {'name': 'danniil'})


In [4]:
dummy_model_instances_list = create_dummy_endovis_records(10)

db.session.add_all(dummy_model_instances_list)
db.session.commit()

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [4]:
experiments = EndovisBinary.query.all()
print(experiments)

first_experiment = experiments[0]

[<Endovis Binary Segmentation experiment id=1, batch_size=100, learning_rate=0.0001>, output_stride=8, graphs=<dash_deep.plot.BaseSegmentationGraph instance at 0x7fad3b0f30e0>, <Endovis Binary Segmentation experiment id=2, batch_size=100, learning_rate=0.0001>, output_stride=8, graphs=<dash_deep.plot.BaseSegmentationGraph instance at 0x7fad3b5ba518>, <Endovis Binary Segmentation experiment id=3, batch_size=100, learning_rate=0.0001>, output_stride=8, graphs=<dash_deep.plot.BaseSegmentationGraph instance at 0x7fad3b2069e0>, <Endovis Binary Segmentation experiment id=4, batch_size=100, learning_rate=0.0001>, output_stride=8, graphs=<dash_deep.plot.BaseSegmentationGraph instance at 0x7fad3b396290>, <Endovis Binary Segmentation experiment id=5, batch_size=100, learning_rate=0.0001>, output_stride=8, graphs=<dash_deep.plot.BaseSegmentationGraph instance at 0x7fad3b0dc368>]
